In [111]:
# General imports
import numpy as np
import torch

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from sklearn.linear_model import BayesianRidge

import seaborn as sns
sns.set()

# Making data

In [112]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)

y = dataset.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1)) # observations
X = dataset.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), poly_order=2, deriv_order=3) # covariates

print(y.shape, X.shape)

(5000, 1) (5000, 12)


In [113]:
y += np.std(y) * 0.5 * np.random.randn(*y.shape)

In [114]:
np.std(y) * 0.5

0.08933559145075419

In [115]:
X = X / np.linalg.norm(X, axis=0, keepdims=True)

# Baseline

In [116]:
reg = BayesianRidge(alpha_1=0, alpha_2=0, lambda_1=0, lambda_2=0, fit_intercept=False, compute_score=True)

In [117]:
reg.fit(X, y)

/home/gert-jan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


BayesianRidge(alpha_1=0, alpha_2=0, compute_score=True, fit_intercept=False,
              lambda_1=0, lambda_2=0)

In [118]:
reg.coef_[:, None]

array([[-0.15504377],
       [-0.63589557],
       [ 7.04794524],
       [-0.17541755],
       [ 0.42338914],
       [-9.49679341],
       [-0.10483916],
       [ 0.83393106],
       [-0.5301025 ],
       [-0.36535585],
       [ 0.06563282],
       [-0.65920726]])

In [119]:
reg.lambda_

0.08105813835726926

In [123]:
np.sqrt(1 / reg.alpha_)

0.08071557742563541

In [121]:
reg.scores_

array([3449.51615418, 5455.4724688 , 5455.93969817, 5455.94012097,
       5455.94012156, 5455.94012841])

# Gradient descent through pytorch

In [14]:
def bayesian_ridge(X, y, alpha_, beta_):
    n_samples = X.shape[0]
    alpha = alpha_ * torch.ones(X.shape[1])
    
    A_inv = torch.inverse(torch.diag(alpha) + beta_ * X.T @ X)
    mn = beta_ * A_inv @ X.T @ y
    E = beta_ * torch.sum((y - X @ mn)**2) + (alpha[:, None] * mn**2).sum()
    
    p_reg = E - (torch.logdet(A_inv) + n_samples * torch.log(beta_) + torch.sum(torch.log(alpha))) # we use alpha and lambda since these are bounded
    return p_reg, mn, A_inv

In [15]:
# Now let's optimize
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

a = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32))
b = torch.nn.Parameter(-torch.log(torch.var(y)))

optimizer = torch.optim.Adam([a, b], lr=1e-2)
max_epochs=1e4

In [16]:
for epoch in torch.arange(max_epochs):
    alpha_ = torch.exp(a).clamp(max=1e8)
    beta_ = torch.exp(b).clamp(max=1e8)
    loss = bayesian_ridge(X, y, alpha_, beta_)[0]
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss)

tensor(-16068.9326, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4141, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)
tensor(-20114.4160, grad_fn=<SubBackward0>)


In [17]:
torch.exp(b)

tensor(153.8698, grad_fn=<ExpBackward>)

In [18]:
torch.exp(a)

tensor([0.0875], grad_fn=<ExpBackward>)

In [19]:
bayesian_ridge(X, y, alpha_, beta_)[1]

tensor([[ 0.1726],
        [-0.6785],
        [ 7.0208],
        [-0.3478],
        [-0.5703],
        [-8.7699],
        [ 0.2836],
        [ 0.8696],
        [ 0.6079],
        [-1.5007],
        [ 0.3572],
        [-0.7488]], grad_fn=<MmBackward>)

Which is the same :)

In [21]:
%%time
bayesian_ridge(X, y, alpha_, beta_)

CPU times: user 10.8 ms, sys: 193 µs, total: 11 ms
Wall time: 933 µs


(tensor(-20114.4160, grad_fn=<SubBackward0>),
 tensor([[ 0.1726],
         [-0.6785],
         [ 7.0208],
         [-0.3478],
         [-0.5703],
         [-8.7699],
         [ 0.2836],
         [ 0.8696],
         [ 0.6079],
         [-1.5007],
         [ 0.3572],
         [-0.7488]], grad_fn=<MmBackward>),
 tensor([[ 1.4191e-02, -4.2949e-03, -1.5426e-02, -6.1098e-03, -2.4818e-02,
           6.8010e-03,  4.4833e-02,  1.7033e-03,  2.4154e-02, -1.0244e-02,
          -2.2387e-02,  1.1025e-03],
         [-4.2948e-03,  1.2603e-01,  1.2305e-02, -2.1856e-02, -1.1560e-03,
          -2.8206e-01,  6.3731e-02, -4.9729e-02,  1.4844e-02,  1.6781e-01,
          -7.0673e-02,  6.5450e-02],
         [-1.5426e-02,  1.2305e-02,  1.6215e-01,  1.1285e-01,  2.3621e-02,
           4.9910e-02, -4.8065e-01, -9.1944e-02, -5.3215e-02, -2.4892e-02,
           3.1999e-01, -5.9870e-04],
         [-6.1100e-03, -2.1856e-02,  1.1285e-01,  1.7461e-01,  9.9012e-03,
           1.0605e-01, -3.4136e-01, -2.3736e-01, -3.58

In [65]:
reg.scores_

array([3440.17045922, 5462.18384077, 5462.51501708, 5462.51507218,
       5462.5150722 , 5462.51507272])

# Direct of marginalization

In [41]:
N = X.shape[0]
M = X.shape[1]

In [64]:
%%time
C = beta_**-1 * torch.eye(N) + alpha_**-1 * X @ X.T
p =-1/2 * (N * np.log(2*np.pi) + torch.logdet(C) + y.T @ torch.inverse(C) @ y)

CPU times: user 18.1 s, sys: 400 ms, total: 18.5 s
Wall time: 1.16 s


In [66]:
p

tensor([[5462.3301]], grad_fn=<MulBackward0>)

In [67]:
%%time
C = beta_**-1 * torch.eye(N) + alpha_**-1 * X @ X.T
p = torch.logdet(C) + y.T @ torch.inverse(C) @ y

CPU times: user 18.1 s, sys: 389 ms, total: 18.5 s
Wall time: 1.16 s


In [68]:
p

tensor([[-20114.0469]], grad_fn=<AddBackward0>)

In [69]:
# Now lets use woodbury

In [40]:
%%time


C_inv = beta_ * (torch.eye(N) - X @ torch.inverse(alpha / beta_ + X.T @ X) @ X.T)

p = - torch.logdet(C_inv) + y.T @ C_inv @ y

NameError: name 'M' is not defined

In [71]:
p

tensor([[-20114.4570]], grad_fn=<AddBackward0>)

In [101]:
%%time
# Optimized version
alpha = alpha_ * torch.eye(M)
gram = X.T @ X
sigma = torch.inverse(alpha / beta_ + gram)
log_det_C_inv = torch.logdet(torch.eye(M) -  gram @ sigma)
p = - N * torch.log(beta_) - log_det_C_inv + beta_ * (y.T @ y - y.T @ X @ sigma @ X.T @ y)

CPU times: user 10.4 ms, sys: 183 µs, total: 10.6 ms
Wall time: 1.02 ms


In [108]:
def bayesian_ridge_direct(X, y, alpha_, beta_):
    N, M = X.shape[0], X.shape[1]
    
    gram = X.T @ X
    sigma = torch.inverse((alpha_ / beta_) * torch.eye(M) + gram)
    mn = sigma @ X.T @ y
    
    p = - N * torch.log(beta_) - torch.logdet(torch.eye(M) -  gram @ sigma) + beta_ * (y.T @ y - y.T @ X @ mn)
    return p, mn

In [110]:
%%time
bayesian_ridge_direct(X, y, alpha_, beta_)

CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 969 µs


(tensor([[-20081.8301]], grad_fn=<AddBackward0>),
 tensor([[-0.0169],
         [-0.3715],
         [ 7.0453],
         [ 0.0754],
         [-0.4383],
         [-8.9243],
         [ 0.3066],
         [ 0.3772],
         [ 0.6063],
         [-1.5424],
         [ 0.3798],
         [-0.6396]], grad_fn=<MmBackward>))

In [104]:
# Now let's optimize
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

a = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32))
b = torch.nn.Parameter(-torch.log(torch.var(y)))

optimizer = torch.optim.Adam([a, b], lr=1e-2)
max_epochs=1e4

<ipython-input-104-37c047237d63>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
<ipython-input-104-37c047237d63>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


In [105]:
for epoch in torch.arange(max_epochs):
    alpha_ = torch.exp(a).clamp(max=1e8)
    beta_ = torch.exp(b).clamp(max=1e8)
    loss = bayesian_ridge_direct(X, y, alpha_, beta_)[0]
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss)

tensor([[-16122.0713]], grad_fn=<AddBackward0>)
tensor([[-20081.6934]], grad_fn=<AddBackward0>)
tensor([[-20081.6523]], grad_fn=<AddBackward0>)
tensor([[-20081.6973]], grad_fn=<AddBackward0>)
tensor([[-20081.4922]], grad_fn=<AddBackward0>)
tensor([[-20081.6133]], grad_fn=<AddBackward0>)
tensor([[-20081.6016]], grad_fn=<AddBackward0>)
tensor([[-20081.8086]], grad_fn=<AddBackward0>)
tensor([[-20081.7441]], grad_fn=<AddBackward0>)
tensor([[-20081.7324]], grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [37]:
alpha_

tensor([0.0880], grad_fn=<ClampBackward>)

In [38]:
beta_

tensor(152.8511, grad_fn=<ClampBackward>)